In [1]:
import torch
from torch.utils.data import TensorDataset, DataLoader
import pandas as pd
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from sklearn.model_selection import train_test_split
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR

In [2]:
## Import and split the training and testing dataset
test_data = pd.read_csv(r'C:\Users\liuch\Desktop\395\Final_Project\Spaceship_Titanic\spaceship-titanic\test_data_featured.csv')

# Change this comment if you want to pick from mode imputation or knn imputation
train_data_mode = pd.read_csv(r'C:\Users\liuch\Desktop\395\Final_Project\Spaceship_Titanic\spaceship-titanic\train_data_mode.csv')
## train_data_mode = pd.read_csv(r'C:\Users\liuch\Desktop\395\Final_Project\Spaceship_Titanic\spaceship-titanic\train_data_knn.csv')

train_data_mode.drop(['PassengerId', 'Name', 'Cabin'], axis=1, inplace=True)
test_data.drop(['PassengerId', 'Name', 'Cabin'], axis=1, inplace=True)

train_mode_y = train_data_mode['Transported']
train_mode_X = train_data_mode.drop('Transported',axis=1)
test_data_y = test_data['Transported']
test_data_X = test_data.drop('Transported',axis=1)

In [3]:
# Convert Boolean Columns to Floats
for col in train_mode_X.columns:
    if train_mode_X[col].dtype == bool:
        train_mode_X[col] = train_mode_X[col].astype(float)

print(train_mode_X.dtypes)

CryoSleep                    float64
Age                          float64
VIP                          float64
RoomService                  float64
FoodCourt                    float64
ShoppingMall                 float64
Spa                          float64
VRDeck                       float64
Num                          float64
Deck_A                       float64
Deck_B                       float64
Deck_C                       float64
Deck_D                       float64
Deck_E                       float64
Deck_F                       float64
Deck_G                       float64
Deck_T                       float64
Side_P                       float64
Side_S                       float64
Destination_55 Cancri e      float64
Destination_PSO J318.5-22    float64
Destination_TRAPPIST-1e      float64
HomePlanet_Earth             float64
HomePlanet_Europa            float64
HomePlanet_Mars              float64
Cabin_encoded                  int64
dtype: object


In [4]:
# Convert Boolean Columns to Floats
for col in test_data_X.columns:
    if test_data_X[col].dtype == bool or test_data_X[col].dtype == object:
        test_data_X[col] = test_data_X[col].astype(float)

print(test_data_X.dtypes)

CryoSleep                    float64
Age                          float64
VIP                          float64
RoomService                  float64
FoodCourt                    float64
ShoppingMall                 float64
Spa                          float64
VRDeck                       float64
Num                          float64
Deck_A                       float64
Deck_B                       float64
Deck_C                       float64
Deck_D                       float64
Deck_E                       float64
Deck_F                       float64
Deck_G                       float64
Deck_T                       float64
Side_P                       float64
Side_S                       float64
Destination_55 Cancri e      float64
Destination_PSO J318.5-22    float64
Destination_TRAPPIST-1e      float64
HomePlanet_Earth             float64
HomePlanet_Europa            float64
HomePlanet_Mars              float64
Cabin_encoded                  int64
dtype: object


In [5]:
# Convert pandas dataframes to numpy arrays and then to PyTorch tensors
train_mode_X_tensor = torch.tensor(train_mode_X.values.astype(np.float32))
train_mode_y_tensor = torch.tensor(train_mode_y.values.astype(np.float32))
test_data_X_tensor = torch.tensor(test_data_X.values.astype(np.float32))
test_data_y_tensor = torch.tensor(test_data_y.values.astype(np.float32))

# Create Tensor datasets
train_dataset = TensorDataset(train_mode_X_tensor, train_mode_y_tensor)
test_dataset = TensorDataset(test_data_X_tensor, test_data_y_tensor)

In [8]:
class BinaryClassifier(nn.Module):
    def __init__(self, input_size):
        super(BinaryClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, 1)

    def forward(self, x):
        # Flatten the input tensor
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, p=0.2)
        x = F.relu(self.fc2(x))
        x = F.dropout(x, p=0.2)
        x = F.relu(self.fc3(x))
        x = torch.sigmoid(self.fc4(x))
        return x

# Create the model instance with the correct input size
input_size = train_mode_X_tensor.shape[1]
model = BinaryClassifier(input_size)


In [11]:
model = BinaryClassifier(input_size)
criterion = nn.BCELoss()
optimizer = Adam(model.parameters(), lr=0.001)
epochs = 500
batch_size = 32
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Training loop
for epoch in range(epochs):
    for inputs, labels in train_loader:
        # Move inputs and labels to the device
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch+1}/{epochs} Loss: {loss.item()}')

Epoch 1/500 Loss: 0.8033303022384644
Epoch 2/500 Loss: 0.53824383020401
Epoch 3/500 Loss: 0.6730542182922363
Epoch 4/500 Loss: 0.5004540085792542
Epoch 5/500 Loss: 0.4031817615032196
Epoch 6/500 Loss: 0.6808675527572632
Epoch 7/500 Loss: 0.5225037932395935
Epoch 8/500 Loss: 0.4470587372779846
Epoch 9/500 Loss: 0.4689681828022003
Epoch 10/500 Loss: 0.4952312707901001
Epoch 11/500 Loss: 0.6034083366394043
Epoch 12/500 Loss: 0.37474751472473145
Epoch 13/500 Loss: 0.5920603275299072
Epoch 14/500 Loss: 0.4045535922050476
Epoch 15/500 Loss: 0.4223123788833618
Epoch 16/500 Loss: 0.4185619354248047
Epoch 17/500 Loss: 0.6873879432678223
Epoch 18/500 Loss: 0.5560809373855591
Epoch 19/500 Loss: 0.7511594891548157
Epoch 20/500 Loss: 0.4917945861816406
Epoch 21/500 Loss: 0.7659285664558411
Epoch 22/500 Loss: 0.3558990955352783
Epoch 23/500 Loss: 0.5413002967834473
Epoch 24/500 Loss: 0.43038687109947205
Epoch 25/500 Loss: 0.46577996015548706
Epoch 26/500 Loss: 0.5433860421180725
Epoch 27/500 Loss: 0

In [13]:
test_batch_size = 32

test_loader = DataLoader(dataset=test_dataset, batch_size=test_batch_size)

model.eval()

correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs)
        predicted = outputs.squeeze().round()
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {100 * correct / total}%')


Accuracy: 65.40894972966755%
